In [1]:
import torch

## 自动求导



### 标量求导

最简单的自动求导。

<br>

求导，只需要将x赋值然后y进行运算，即可用PyTorch获取导数。
$$y=(1-x)^2$$

$$
\frac{\mathrm{d} y}{\mathrm{d} x}
= \frac{\mathrm{d}}{\mathrm{d} x} (1-x)^2
= 2x - 2
$$

In [2]:
'''
第一步是给x赋值一个需要求导的标量
需要注意的是如果要保存梯度则只能传入浮点数
requires_grad参数是用来存储梯度的
'''
x = torch.tensor(4.0, requires_grad=True)
x

tensor(4., requires_grad=True)

In [3]:
# 直接把求导的表达式扔进去
y = (1 - x) ** 2

In [4]:
# x关于y的导数轻松求出。
y.backward()
x.grad

tensor(6.)

## 向量对标量求导
<br><br><br>
对函数$y = 2x^Tx$

列向量x求导

In [5]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [6]:
# 梯度存储
x.requires_grad_(True)
# 或者创建矩阵的时候加参数也可以
# x = torch.arange(4.0, requires_grad=True)
print(x.grad) #默认值为None

None


y为自身点积的两倍,表达式为

$$
y = 2x^Tx = 2(x_0^2+x_1^2+x_2^2+x_3^2+...+x_n^2)
$$

对其求偏导

In [7]:
# 导入导数的表达式
y = 2 * torch.dot(x, x)

In [8]:
# 求导
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

对其所有元素做偏导，表达式相同：以$x_0$为例:

$$
\frac{\partial y}{\partial x_0}=
\frac{\partial}{\partial x_0}(2x^Tx)=
\frac{\partial}{\partial x_0}2(x_0^2+x_1^2+x_2^2+x_3^2+...+x_n^2)=
4x
$$

In [9]:
# 验证一下
x.grad == 4 * x

tensor([True, True, True, True])

In [10]:
# 默认累积梯度，需要清除
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

当y计算出不是标量时，两个向量的导数就是矩阵。

下面的例子目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [11]:
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# y先做求和再求导，实际上还是标量和向量的求导
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算挪出计算图

In [12]:
x.grad.zero_()
y = x * x
# y加了detach函数后将计算图分离了，u变成了单纯的常数
u = y.detach()
z = u * x

# 用z做x的偏导数
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

重置x的梯度后，用y做反向传播，则得到$y = x^2$的导数，即$2x$

In [13]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

用Python控制调用

In [14]:
def f(a):
    b = a * 2

    while b.norm() < 1000:
        b = b * 2

    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [15]:
#取随机数的标量
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

f函数内的操作全部都是线性的。所以可以直接等于d/a

In [16]:
a.grad == d / a

tensor(True)